In [1]:
import pandas as pd 
import scanpy as sc
import phenograph
import parc
from utag import utag
import os 
import numpy as np
import time
import anndata as ad

os.chdir("/home/abombin/Mawa/multiplex-analysis-web-apps")

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("input/measurementsthymus-exported.csv")
list(df.columns)
meta = df.iloc[:, :4]
list(meta.columns)
mat = df.iloc[:, 4:]
list(mat.columns)
meta = meta.rename(columns={"Image": "Image ID_(standardized)", 
                            "Centroid X µm": "Centroid X (µm)_(standardized)", 
                            "Centroid Y µm": "Centroid Y (µm)_(standardized)"})

adata = ad.AnnData(mat)
adata.obs = meta

adata.obsm['spatial'] = np.array(adata.obs[["Centroid X (µm)_(standardized)", "Centroid Y (µm)_(standardized)"]])


/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
%%time
utag_results = utag(adata,
        slide_key="Image ID_(standardized)",
        normalization_mode='l1_norm',
        apply_clustering=True,
        clustering_method = "leiden", 
        resolutions = [1],
        leiden_kwargs={"n_iterations": -1, "random_state": 42}
    )

In [3]:

sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.pca(adata, n_comps=10)

adata

AnnData object with n_obs × n_vars = 1274717 × 20
    obs: 'Image ID_(standardized)', 'Class', 'Centroid X (µm)_(standardized)', 'Centroid Y (µm)_(standardized)'
    uns: 'log1p', 'pca'
    obsm: 'spatial', 'X_pca'
    varm: 'PCs'

In [5]:
%%time
utag_results = utag(adata,
        slide_key="Image ID_(standardized)",
        normalization_mode=None,
        apply_clustering=True,
        clustering_method = "kmeans", 
        resolutions = [1], parallel=True,
        processes=8
    )

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Applying UTAG Algorithm...


  0%|          | 0/1 [00:00<?, ?it/s]

Applying K-means Clustering at Resolution: 1...


100%|██████████| 1/1 [00:04<00:00,  5.00s/it]

CPU times: user 4min 4s, sys: 19.8 s, total: 4min 24s
Wall time: 4min 4s


In [7]:
adata
utag_results

AnnData object with n_obs × n_vars = 1274717 × 20
    obs: 'Image ID_(standardized)', 'Class', 'Centroid X (µm)_(standardized)', 'Centroid Y (µm)_(standardized)', 'UTAG Label_kmeans_1'
    uns: 'pca', 'neighbors'
    obsm: 'spatial', 'X_pca', 'UTAG Label_kmeans_1_probabilities'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [4]:
%%time
utag_results = utag(adata,
        slide_key="Image ID_(standardized)",
        normalization_mode='l1_norm',
        apply_clustering=True,
        clustering_method = "kmeans",
        resolutions = [1], parallel=False,
        processes=1
    )

/home/abombin/miniconda3/envs/mawa-2024-02-08/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Applying UTAG Algorithm...


  0%|          | 0/4 [00:00<?, ?it/s]

MemoryError: Unable to allocate 2.54 TiB for an array with shape (590517, 590517) and data type float64